In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd My\ Drive

/content/drive/My Drive


In [4]:
cd OurCode/

/content/drive/My Drive/OurCode


In [5]:
import re
import pandas as pd
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

#For Tweet Processing
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences

#For Keras Model
from keras.models import Model
from keras.models import Sequential
from keras.models import load_model

#For CNN Layers
from keras.layers import Embedding, LSTM, Dense, Bidirectional

from keras.optimizers import Adam

from sklearn.utils import shuffle

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Using TensorFlow backend.


In [6]:
data = pd.read_csv("BigTraining.csv",header=None)
data.head()

,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
del data[1],data[2],data[3],data[4]
data.head()

,0,5
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [8]:
data[0] = data[0].apply(lambda x: 1 if x == 4 else 0)
data.columns = ["sentiment","tweet"]
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,sentiment,tweet
0,0,RIP passengers/crew of AirFrance 447 http:/...
1,1,@ningningning I just got the 29.90 6 drinks pa...
2,1,@psynister agreed. it's working out a lot bett...
3,0,The rush for 3.x has been a /little/ more than...
4,1,@Sarah_1991 swine flu could be how the world e...


In [0]:
#####################################################
################# Utility Functions #################
#####################################################

def replaceUserName(tweet):
	tweet = re.sub('@[^\s]+','USERNAME',tweet)
	return tweet

def replaceLink(tweet):
	tweet = re.sub('((www\.[^\s]+)|((https|http)?://[^\s]+))','URL',tweet)
	return tweet

def replaceWithTwoChar(tweet):
    pattern = re.compile(r"(.)\1{1,}") 
    tweet = pattern.sub(r"\1\1", tweet)
    return tweet
  
stopWords = list(set(stopwords.words("english")))
stopWords = [str(word) for word in stopWords]
MAXLEN = float('-inf')

def cleanTweet(tweet):
  global MAXLEN
  tweet = tweet.strip().lower()
  tweet = replaceUserName(tweet)
  tweet = replaceLink(tweet)
  tweet = replaceWithTwoChar(tweet)
  wordsInTweet = tweet.split()
  #Removing StopWords to reduce no. of features
  #text = ' '.join([word for word in wordsInTweet if word not in stopWords])
  #wordsInTweet =  text.split()

  if MAXLEN < len(wordsInTweet):
    MAXLEN = len(wordsInTweet)
  return wordsInTweet

In [10]:
data['tokens'] = data['tweet'].apply(lambda x: cleanTweet(x))
del data['tweet']
data.head()

,sentiment,tokens
0,0,"[rip, passengers/crew, of, airfrance, 447, URL]"
1,1,"[USERNAME, i, just, got, the, 29.90, 6, drinks..."
2,1,"[USERNAME, agreed., it's, working, out, a, lot..."
3,0,"[the, rush, for, 3.x, has, been, a, /little/, ..."
4,1,"[USERNAME, swine, flu, could, be, how, the, wo..."


In [0]:
MAX_NB_WORDS = 90000
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
total_size = 800000
test_total_size = 150000
tokenizer.fit_on_texts(data['tokens'][:total_size])

In [12]:
#Colab not able to handle processing all at once
batch_size = total_size/100
number_of_batches = 100
train_seq = list()
for i in range(number_of_batches):
  print i+1,"batch"
  batch_seq = tokenizer.texts_to_sequences(data["tokens"][i*batch_size:(i+1)*batch_size])
  train_seq = np.concatenate((train_seq,batch_seq))
del batch_seq

1 batch
2 batch
3 batch
4 batch
5 batch
6 batch
7 batch
8 batch
9 batch
10 batch
11 batch
12 batch
13 batch
14 batch
15 batch
16 batch
17 batch
18 batch
19 batch
20 batch
21 batch
22 batch
23 batch
24 batch
25 batch
26 batch
27 batch
28 batch
29 batch
30 batch
31 batch
32 batch
33 batch
34 batch
35 batch
36 batch
37 batch
38 batch
39 batch
40 batch
41 batch
42 batch
43 batch
44 batch
45 batch
46 batch
47 batch
48 batch
49 batch
50 batch
51 batch
52 batch
53 batch
54 batch
55 batch
56 batch
57 batch
58 batch
59 batch
60 batch
61 batch
62 batch
63 batch
64 batch
65 batch
66 batch
67 batch
68 batch
69 batch
70 batch
71 batch
72 batch
73 batch
74 batch
75 batch
76 batch
77 batch
78 batch
79 batch
80 batch
81 batch
82 batch
83 batch
84 batch
85 batch
86 batch
87 batch
88 batch
89 batch
90 batch
91 batch
92 batch
93 batch
94 batch
95 batch
96 batch
97 batch
98 batch
99 batch
100 batch


In [0]:
MAXLEN += 10
padded_train_sequences = pad_sequences(train_seq, maxlen=MAXLEN)
del train_seq

In [0]:
y_train = data['sentiment'][:total_size]
y_test = data['sentiment'][total_size:total_size+test_total_size]

In [15]:
batch_size = test_total_size/50
number_of_batches = 50
test_seq = list()
for i in range(number_of_batches):
  print i+1,"batch"
  batch_seq = tokenizer.texts_to_sequences(data["tokens"][total_size+i*batch_size:total_size+(i+1)*batch_size])
  test_seq = np.concatenate((test_seq,batch_seq))
del batch_seq

1 batch
2 batch
3 batch
4 batch
5 batch
6 batch
7 batch
8 batch
9 batch
10 batch
11 batch
12 batch
13 batch
14 batch
15 batch
16 batch
17 batch
18 batch
19 batch
20 batch
21 batch
22 batch
23 batch
24 batch
25 batch
26 batch
27 batch
28 batch
29 batch
30 batch
31 batch
32 batch
33 batch
34 batch
35 batch
36 batch
37 batch
38 batch
39 batch
40 batch
41 batch
42 batch
43 batch
44 batch
45 batch
46 batch
47 batch
48 batch
49 batch
50 batch


In [16]:
padded_test_sequences = pad_sequences(test_seq, maxlen=MAXLEN)
del test_seq
print len(padded_test_sequences),len(padded_test_sequences[0])

150000 74


In [0]:
embedding_dim = 300
filter_sizes = [4, 4, 5]
num_filters = 256
drop = 0.5
embedding_matrix = np.random.random((MAX_NB_WORDS, embedding_dim))


In [0]:
rnnModel = Sequential()
rnnModel.add(Embedding(MAX_NB_WORDS, embedding_dim, input_length=MAXLEN))
rnnModel.add(Bidirectional(LSTM(MAXLEN)))
rnnModel.add(Dense(units=1,activation="sigmoid"))
rnnModel.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [19]:
rnnModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 74, 300)           27000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 148)               222000    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 149       
Total params: 27,222,149
Trainable params: 27,222,149
Non-trainable params: 0
_________________________________________________________________


In [20]:
batch_size = 256
epochs = 4
history = rnnModel.fit(x=padded_train_sequences, 
                    y=y_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    verbose=1)

Epoch 1/4
800000/800000 [==============================] - 1222s 2ms/step - loss: 0.4251 - acc: 0.8023
Epoch 2/4
800000/800000 [==============================] - 1214s 2ms/step - loss: 0.3426 - acc: 0.8483
Epoch 3/4
800000/800000 [==============================] - 1218s 2ms/step - loss: 0.2749 - acc: 0.8821
Epoch 4/4
800000/800000 [==============================] - 1226s 2ms/step - loss: 0.2101 - acc: 0.9119


In [28]:
y_pred_rnn = rnnModel.predict(padded_test_sequences, verbose=1, batch_size=2048)

y_pred_rnn = pd.DataFrame(y_pred_rnn, columns=['prediction'])
print y_pred_rnn.head()
#y_pred_rnn['prediction'] = y_pred_rnn['prediction'].map(lambda p: 1 if p >= 0.5 else 0)
#y_pred_cnn_multi_channel.to_csv('./predictions/y_pred_cnn_multi_channel.csv', index=False)

150000/150000 [==============================] - 14s 95us/step
   prediction
0    0.999260
1    0.005495
2    0.021846
3    0.734699
4    0.999999


In [22]:
from sklearn.metrics import accuracy_score
#y_pred_cnn_multi_channel = pd.read_csv('./predictions/y_pred_cnn_multi_channel.csv')
print(accuracy_score(y_test, y_pred_rnn))

0.8071266666666667


In [23]:
t = "I am very happy"
s = []
t = cleanTweet(t)
print t

['i', 'am', 'very', 'happy']


In [24]:
s.append(t)
t = tokenizer.texts_to_sequences(s)
t = pad_sequences(t,maxlen=MAXLEN)
t

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   2,  59, 108, 116]], dtype=int32)

In [25]:
rnnModel.predict(t, verbose=1)

1/1 [==============================] - 0s 137ms/step


array([[0.99059033]], dtype=float32)

In [0]:
rnnModel.save("LSTM_Bidirectional_MAXLEN.h5")

In [0]:
import pickle

with open('rnn_tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [30]:
i = 0
for index,row in y_pred_rnn.iterrows():
  if row['prediction'] >= 0.5:
    temp = 1
  else:
    temp = 0
  if temp != y_test[total_size+index]:
    i += 1
    print row['prediction'],y_test[total_size+index]
    print data['tokens'][total_size+index]
    if i == 20:
      break

0.7346993 0
['USERNAME', 'yea,', 'its', 'just', 'one', 'of', 'those', 'days.']
0.95094067 0
['nothing,', 'sweet', 'nothing']
0.01674008 1
['USERNAME', 'i', 'dont', 'feel', 'the', 'haze', 'at', 'all.', 'maybe', 'ur', 'area', 'is', 'too', 'polluted.']
0.49373412 1
['USERNAME', 'i', 'wish', 'i', 'could', 'take', 'credit', 'for', 'that', 'but', "i'm", 'paraphrasing', 'my', 'little', 'cousin.', "she's", 'brilliant']
0.5661671 0
['almost', 'time', 'for', 'school.', 'we', 'have', 'the', 'oat', 'today.']
0.35051504 1
['USERNAME', 'that', 'is', 'me!']
0.46621445 1
['USERNAME', 'swiss', 'cheese?', 'that', 'means', 'your', 'brain', 'is', 'very', 'tasty', 'on', 'toast', 'with', 'avocado', 'and', 'sliced', 'turkey.']
0.42829412 1
['USERNAME', 'jk', 'rowling', 'writes', 'books,', "doesn't", 'make', 'her', 'a', 'journalist!']
0.54496986 0
["it's", 'windy', 'outside.', 'i', 'wanna', 'go', 'to', "sonya's", 'garden!']
0.9318707 0
['i', "don't", 'feel', 'good', 'wow.', 'when', 'it', 'rains', 'on', 'my', 